# ComfyUI Launcher

## Env

In [ ]:
# set proxy

#!export HTTP_PROXY=http://host.docker.internal:1081
#!export HTTPS_PROXY=$HTTP_PROXY
#!export http_proxy=$HTTP_PROXY
#!export https_proxy=$HTTP_PROXY
#!export NO_PROXY="localhost,*.local,*.internal,[::1],fd00::/7,
# 10.0.0.0/8,127.0.0.0/8,169.254.0.0/16,172.16.0.0/12,192.168.0.0/16,
# 10.*,127.*,169.254.*,172.16.*,172.17.*,172.18.*,172.19.*,172.20.*,
# 172.21.*,172.22.*,172.23.*,172.24.*,172.25.*,172.26.*,172.27.*,
# 172.28.*,172.29.*,172.30.*,172.31.*,172.32.*,192.168.*,
# *.cn,ghproxy.com,*.ghproxy.com,ghproxy.org,*.ghproxy.org,
# gh-proxy.com,*.gh-proxy.com,ghproxy.net,*.ghproxy.net"
#!export no_proxy=$NO_PROXY
#!echo "[INFO] Proxy set to $HTTP_PROXY"

# os.environ['HTTP_PROXY'] = 'http://host.docker.internal:1081'
# os.environ['HTTPS_PROXY'] = os.environ['HTTP_PROXY']
# os.environ['http_proxy'] = os.environ['HTTP_PROXY']
# os.environ['https_proxy'] = os.environ['HTTP_PROXY']
# os.environ['NO_PROXY'] = "localhost,*.local,*.internal,[::1],fd00::/7,10.0.0.0/8,127.0.0.0/8,169.254.0.0/16,172.16.0.0/12,192.168.0.0/16,10.*,127.*,169.254.*,172.16.*,172.17.*,172.18.*,172.19.*,172.20.*,172.21.*,172.22.*,172.23.*,172.24.*,172.25.*,172.26.*,172.27.*,172.28.*,172.29.*,172.30.*,172.31.*,172.32.*,192.168.*,*.cn,ghproxy.com,*.ghproxy.com,ghproxy.org,*.ghproxy.org,gh-proxy.com,*.gh-proxy.com,ghproxy.net,*.ghproxy.net"
# os.environ['no_proxy'] = os.environ['NO_PROXY']
# print(f"🟡 Proxy set to {os.environ['HTTP_PROXY']}")

In [ ]:
# set proxy (autodl)
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"',
#                         shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [ ]:
%pip install toml tqdm
import os
import shutil
import time
import toml
import subprocess
from urllib.parse import urlparse

# global utils
def is_dir_empty(path):
    return len(os.listdir(path)) == 0

In [ ]:
# config
## path
WORKSPACE_PATH = os.path.join('..','workspace')
WORKSPACE_PATH = os.path.abspath(WORKSPACE_PATH)
MODEL_PATH = os.path.join(WORKSPACE_PATH, 'models')
APP_PATH = os.path.join(WORKSPACE_PATH, 'comfyui')

## custom nodes init
CUSTOM_NODES_CONFIG = 'comfyui-custom_nodes.toml'
CUSTOM_NODES_INSTALL_REQ = True

## model downloading
MODELS_DOWNLOAD = True
MODELS_CONFIG = 'comfyui-models.toml'
MODELS_INCLUDE_CATEGORY = ['sdxl']
MODELS_INCLUDE_TAGS = ['set-lite']

## app launch
AUTO_UPDATE = True
LAUNCH_CLI_ARGS = [
    "--listen 127.0.0.1",
    "--port 8080",
    "--disable-auto-launch"
]

In [ ]:
# debug
## debug env
WORKSPACE_PATH = os.path.join('..', 'workspace', 'test')
# use absolute path
WORKSPACE_PATH = os.path.abspath(WORKSPACE_PATH)
MODEL_PATH = os.path.join(WORKSPACE_PATH, 'models')
APP_PATH = os.path.join(WORKSPACE_PATH, 'comfyui')

CUSTOM_NODES_INSTALL_REQ = True
MODELS_DOWNLOAD = True
MODELS_INCLUDE_CATEGORY = ['sd15']
AUTO_UPDATE = True

## Initialize app

In [ ]:
# check if nvidia cuda is available
def check_gpu():
    try:
        print(subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout.decode('utf-8'))
        print("🟢 NVIDIA GPU found")        
        return True
    except FileNotFoundError:   
        print("🟡 NVIDIA GPU not found")
        return False
check_gpu()

In [ ]:
# initialize comfyui
print('📦 Start: initialize comfyui')
try:
    if os.path.exists(APP_PATH) and AUTO_UPDATE:
        print("🔄 Existed comfyui, checking updates...")
        update_result = subprocess.run(['git', 'pull'], cwd=APP_PATH, capture_output=True, text=True, check=True)
        if 'Already up to date' in update_result.stdout:
            print("🔄 comfyui is already up to date!")
        else:
            print("🔄 comfyui is updated!")
    elif os.path.exists(APP_PATH):
        print("📦 Existed comfyui, skip installation")
    else:
        print('📦 Install: comfyui')
        subprocess.run(['git', 'clone', 'https://github.com/comfyanonymous/ComfyUI.git', APP_PATH], check=True)
        if check_gpu():
            subprocess.run(['pip', 'install', 'xformers', '-r', os.path.join(APP_PATH, "requirements.txt")], cwd=APP_PATH, check=True)
        else:
            subprocess.run(['pip', 'install', '-r', os.path.join(APP_PATH, "requirements.txt")], cwd=APP_PATH, check=True)
except Exception as e:
    print(f"🔴 Error: {e}")

## Download models

In [ ]:
# prepare models dir
# !mkdir -p $MODEL_PATH/configs
# !cp -rf $APP_PATH/models/configs/* $MODEL_PATH/configs
# !rm -rf $APP_PATH/models
# !ln -sf $MODEL_PATH $APP_PATH/models
os.makedirs(os.path.join(MODEL_PATH), exist_ok=True)
if not os.path.islink(os.path.join(APP_PATH, "models")):
    src = MODEL_PATH
    dst = os.path.join(APP_PATH, "models")
    if os.path.exists(os.path.join(dst, "configs")) and not is_dir_empty(dst) and not os.path.exists(os.path.join(src, "configs")):
        shutil.copytree(os.path.join(dst, "configs"), os.path.join(src, "configs"), dirs_exist_ok=True)
    shutil.rmtree(os.path.join(dst))
    os.symlink(src, dst, target_is_directory=True)

In [ ]:
# import download function
from scripts.download import download_models

# exec models download
if MODELS_DOWNLOAD:
    config_models = toml.load(MODELS_CONFIG)
    models = config_models['models']
    download_models(models, MODEL_PATH, MODELS_INCLUDE_CATEGORY, MODELS_INCLUDE_TAGS)

## Initialize `custom_nodes`

In [ ]:
def init_custom_nodes(custom_nodes, install_requirements=CUSTOM_NODES_INSTALL_REQ, max_retries=3, retry_interval=5):
    print(f"📦 Start: initializing custom nodes")
    requirements_list = []
    new_list = []
    existing_list = []
    updated_list = []
    error_list = []
    for cn in custom_nodes:
        cn_url = cn['url']
        cn_name = urlparse(cn_url).path.split('/')[-1].split('.')[0]
        cn_path = os.path.join(APP_PATH, "custom_nodes", cn_name)
        if os.path.exists(os.path.join(cn_path, '.git')) and len(os.listdir(cn_path)) == 1:
            print(f"🔴 {cn_name}: custom nodes broken, will reinstall")
            shutil.rmtree(cn_path)
        retries = 0
        while retries < max_retries:
            try:
                if os.path.exists(cn_path) and not is_dir_empty(cn_path):
                    print(f"🟡 {cn_name}: existed custom nodes.")
                    existing_list.append(cn_name)
                    if AUTO_UPDATE:
                        update_result = subprocess.run(['git', 'pull'], cwd=cn_path, capture_output=True, text=True, check=True)
                        if 'Already up to date' in update_result.stdout:
                            print(f"🟡 {cn_name}: custom nodes already up to date.")
                            break
                        else:
                            print(f"🟢 {cn_name}: custom nodes updated.")
                            updated_list.append(cn_name)
                            break
                else:
                    print(f"🟢 {cn_name}: new custom nodes, cloning...")
                    subprocess.check_call(['git', 'clone', cn_url, cn_path])
                    requirements_list + ["-r", os.path.join(cn_path, "requirement.txt")]
                    new_list.append(cn_name)
                    break
            except Exception as e:
                print(f"🔴 {cn_name}: error: {e}")
                retries += 1
                print(f"⏳ Retrying, sleep {retry_interval}s... ({retries}/{max_retries})")
                time.sleep(retry_interval)
        if retries >= max_retries:
            print(f"🔴 {cn_name}: failed to clone!")
            error_list.append(cn_name)
            return False
        
    total_counts = len(custom_nodes)
    existing_counts =len(existing_list)
    new_counts = len(new_list)
    error_counts = len(error_list)
    print(f"📦 Finished: initialize custom nodes")
    print(f"📦 Total: {total_counts}")
    print(f"🟡 Existed: {existing_counts}")
    if AUTO_UPDATE:
        updated = len(updated_list)
        print(f"🟢 Updated: {updated}")
    print(f"🟢 New: {new_counts}")
    if new_counts > 0:
        for i in new_list:
            print(f"- {i}")
    print(f"🔴 Error: {error_counts}")
    if error_counts > 0:
        for i in error_list:
            print(f" - {i}")

    if install_requirements and requirements_list:
        print(f"📦 Try: pre-install new custom nodes requirements")
        install_requirements_command = ["pip", "install"] + requirements_list
        try:
            subprocess.run(install_requirements_command, cwd=APP_PATH, check=True)
        except Exception as e:
            print(f"🔴 Error: {e}")
            return False
        print(f"🟢 Success: pre-install new custom nodes requirements.")

    return True

In [ ]:
# initial comfyui custom_nodes
config_custom_nodes = toml.load(CUSTOM_NODES_CONFIG)
custom_nodes = config_custom_nodes['custom_nodes']
custom_nodes_models = config_custom_nodes['models']
# download custom_nodes from list
init_custom_nodes(custom_nodes)
download_models(custom_nodes_models, MODEL_PATH, MODELS_INCLUDE_CATEGORY, MODELS_INCLUDE_TAGS)

## Launch

In [ ]:
!cd $APP_PATH
!python main.py {LAUNCH_CLI_ARGS}